# Gravity

 This tutorial demonstrates how to simulate gravitational attraction between two particles using the **SOFA** physics engine.

Initialization:

In [ ]:
import Sofa
import SofaRuntime
SofaRuntime.init()

In [ ]:
root = Sofa.Core.Node("root")

In [ ]:
print(f"By default, SOFA gravity is set to {root.gravity.value}")

By default, SOFA applies uniform gravity. Here, we disable it to focus on **non-uniform gravitational forces** between particles.

In [ ]:
root.gravity.value = [0, 0, 0]

We use SOFA's built-in animation loop for time-stepped physics updates.

In [ ]:
root.addObject("DefaultAnimationLoop")

In [ ]:
SofaRuntime.importPlugin("Sofa.Component.ODESolver.Forward")
root.addObject("EulerExplicitSolver")

We create two particles

In [ ]:
SofaRuntime.importPlugin("Sofa.Component.StateContainer")
initial_position = [[0,0], [1,-1]]
initial_velocity = [[0,0], [0,1.5]]
root.addObject("MechanicalObject", template="Vec2", name="particles", 
    position=initial_position, velocity=initial_velocity)

Definition of the mass of the particles:

In [ ]:
mass = [1, 1]

In [ ]:
SofaRuntime.importPlugin("Sofa.Component.Mass")
root.addObject("UniformMass", template="Vec2", name="mass", vertexMass=mass)

We implement a gravitational force between the two particles using Newton’s law of gravitation:

$$
F_0 = G \frac{m_0 m_1}{r^2} \hat{r}
$$

where:

- $G$ is the gravitational constant (6.67430 x 10-11 m3 kg-1 s-2)
- $m_0$, $m_1$ are particle masses
- $\hat{r}$ is the unit vector pointing from particle 1 to particle 0


In [ ]:
import numpy

# gravitational constant
G = 1

class GravityForceField(Sofa.Core.ForceFieldVec2d):

    def addForce(self, m, out_force, pos, vel):

        d = pos[0] - pos[1] # vector from particle 1 to particle 0
        r_2 = numpy.dot(d, d) # squared norm

        d = d / numpy.sqrt(r_2) #make d a unit vector
        
        with out_force.writeableArray() as wa:
            wa[0] += -(G * mass[0] * mass[1] / r_2) * d
            wa[1] += (G * mass[0] * mass[1] / r_2) * d

root.addObject(GravityForceField())

In [ ]:
Sofa.Simulation.initRoot(root)

Run the simulation and record the trajectories:

In [ ]:
def run_simulation(node, number_time_steps):

    x = [[], []]
    y = [[], []]

    def retrieve_position():
        position = node.particles.position.value
        assert len(position) == 2

        x[0].append(position[0][0])
        y[0].append(position[0][1])

        x[1].append(position[1][0])
        y[1].append(position[1][1])


    retrieve_position()

    for iteration in range(number_time_steps):
        Sofa.Simulation.animate(node, node.dt.value)
        retrieve_position()

    return (x, y)

In [ ]:
x,y = run_simulation(root, 500)

Create the animation of the trajectory:

In [ ]:
from matplotlib import pyplot as plt
import matplotlib.animation as animation

def make_animation(x, y):
    fig, ax = plt.subplots()
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_title("Gravity simulation using SOFA")

    # Initialize empty lines for particles
    line0, = ax.plot([], [], 'ro', markersize=8)  # Red circle for particle 0
    line1, = ax.plot([], [], 'b-', linewidth=2)   # Blue line for particle 1

    # Set initial data ranges
    xmin, xmax = min(min(x[0]), min(x[1])), max(max(x[0]), max(x[1]))
    ymin, ymax = min(min(y[0]), min(y[1])), max(max(y[0]), max(y[1]))

    # Add margin
    xmin = xmin - (xmax - xmin) * 0.1
    xmax = xmax + (xmax - xmin) * 0.1
    ymin = ymin - (ymax - ymin) * 0.1
    ymax = ymax + (ymax - ymin) * 0.1

    # Adjust axis limits to show all data
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)

    def animate(i):
        line0.set_xdata(x[0][:i])
        line0.set_ydata(y[0][:i])

        line1.set_xdata(x[1][:i])
        line1.set_ydata(y[1][:i])

        return (line0, line1)

    # Create animation
    ani = animation.FuncAnimation(
        fig=fig,
        func=animate,
        frames=len(x[0]),
        interval=10  # milliseconds between frames
    )
    plt.close()

    return ani


In [ ]:
ani = make_animation(x, y)

Visualize the animation as javascript:

In [ ]:
# from IPython.display import HTML
# html = ani.to_jshtml()
# HTML(html)

Visualize the animation as html video (requires ffmpeg):

In [ ]:
from IPython.display import HTML
video = ani.to_html5_video()
HTML(video)

## Implicit Solver

In [ ]:
import sympy as smp

In [ ]:
x_0, x_1, y_0, y_1 = smp.symbols('x_0 x_1 y_0 y_1')
m_0, m_1 = smp.symbols('m_0 m_1')
G = smp.symbols('G')

In [ ]:
q_0 = smp.Matrix([x_0, y_0])
q_1 = smp.Matrix([x_1, y_1])
q = smp.Matrix([q_0, q_1])

In [ ]:
r = q_0 - q_1

In [ ]:
r

In [ ]:
radius = smp.sqrt(r.T.dot(r))

In [ ]:
radius

In [ ]:
f_1 = G * m_0 * m_1 * r / radius ** 3

In [ ]:
f_0 = -f_1

In [ ]:
f = smp.Matrix([f_0, f_1])

In [ ]:
f

In [ ]:
df_dq = smp.simplify(f.jacobian(smp.Matrix([q_0, q_1])))

In [ ]:
df_dq

In [ ]:
f_num = smp.lambdify((x_0, y_0, x_1, y_1, G, m_0, m_1), f, 'numpy')

In [ ]:
df_dq_num = smp.lambdify((x_0, y_0, x_1, y_1, G, m_0, m_1), df_dq, 'numpy')

In [ ]:
root_implicit = Sofa.Core.Node("root")
root_implicit.gravity.value = [0, 0, 0]
root_implicit.dt.value = 0.01
root_implicit.addObject("DefaultAnimationLoop")

SofaRuntime.importPlugin("Sofa.Component.ODESolver.Backward")
root_implicit.addObject("EulerImplicitSolver")

# SofaRuntime.importPlugin("Sofa.Component.LinearSolver.Direct")
# root_implicit.addObject("EigenSimplicialLDLT", template='CompressedRowSparseMatrixMat3x3')

SofaRuntime.importPlugin("Sofa.Component.LinearSolver.Iterative")
root_implicit.addObject("CGLinearSolver", iterations=1000)


root_implicit.addObject("MechanicalObject", template="Vec2", name="particles", 
    position=[[0,0], [0.1,0]], velocity=[[0,0], [0,0.1]])

mass = [1, 1]
root_implicit.addObject("UniformMass", template="Vec2", name="mass", vertexMass=mass)

In [ ]:
G = 1

class GravityForceField_implicit(Sofa.Core.ForceFieldVec2d):

    def addForce(self, m, out_force, pos, vel):

        force = f_num(pos[0][0], pos[0][1], pos[1][0], pos[1][1], G, mass[0], mass[1])
        self.dforce_dx = df_dq_num(pos[0][0], pos[0][1], pos[1][0], pos[1][1], G, mass[0], mass[1])
        
        with out_force.writeableArray() as wa:
            wa[0][0] += force[0][0]
            wa[0][1] += force[1][0]
            wa[1][0] += force[2][0]
            wa[1][1] += force[3][0]

    def addDForce(self, mp, df, dx):
        k = mp['kFactor']
        with df.writeableArray() as wa:
            flat_dx = numpy.array([dx[0][0], dx[0][1], dx[1][0], dx[1][1]])
            flat_df = k * self.dforce_dx @ flat_dx
            wa[0][0] += flat_df[0]
            wa[0][1] += flat_df[1]
            wa[1][0] += flat_df[2]
            wa[1][1] += flat_df[3]
            
    def addKToMatrix(self, mp, number_of_nodes, number_of_dofs_per_node):
        return numpy.zeros((0, 0, 1))

root_implicit.addObject(GravityForceField_implicit())

In [ ]:
Sofa.Simulation.initRoot(root_implicit)

In [ ]:
x, y = run_simulation(root_implicit, 500)

In [ ]:
ani = make_animation(x, y)

In [ ]:
from IPython.display import HTML
video = ani.to_html5_video()
HTML(video)